In [1]:
import optuna
from surprise import KNNBasic,KNNWithMeans,KNNWithZScore,KNNBaseline, Dataset, Reader
from surprise.model_selection import cross_validate
import numpy as np
np.random.seed(42)
# Define the dataset path and the reader format
file_path = 'book_ratings.dat'  
reader = Reader(line_format='user item rating', sep='\t', rating_scale=(1, 10))

# Load the dataset
data = Dataset.load_from_file(file_path, reader=reader)

In [2]:
# Mapping of algorithm names to classes
algo_mapping = {
    'KNNBasic': KNNBasic,
    'KNNWithMeans': KNNWithMeans,
    'KNNWithZScore': KNNWithZScore,
    'KNNBaseline': KNNBaseline,
    
}

# List of algorithms to evaluate
algorithms = ['KNNBasic', 'KNNWithMeans', 'KNNWithZScore', 'KNNBaseline']

# Dictionary to store the results
results = {}

# Evaluate each algorithm
for algo_name in algorithms:
    algo_class = algo_mapping[algo_name]  # Get the class from the name
    algo = algo_class()  # Instantiate the algorithm with default parameters
    # Perform cross-validation and store results
    cv_results = cross_validate(algo, data, measures=['RMSE', 'MAE', 'FCP'], cv=5, verbose=False)
    # Compute the average RMSE, MAE, and FCP across the folds
    avg_rmse = np.mean(cv_results['test_rmse'])
    avg_mae = np.mean(cv_results['test_mae'])
    avg_fcp = np.mean(cv_results['test_fcp'])
    results[algo_name] = {
        'RMSE': avg_rmse,
        'MAE': avg_mae,
        'FCP': avg_fcp
    }

# Assuming results is a dictionary containing the performance of each algorithm
with open('algorithm_performance.txt', 'w') as file:
    for algo_name in results:
        file.write(f"{algo_name}: RMSE: {results[algo_name]['RMSE']}, "
                   f"MAE: {results[algo_name]['MAE']}, "
                   f"FCP: {results[algo_name]['FCP']}\n")

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [3]:
# Define the objective function for hyperparameter optimization for KNNBasic
def knn_basic_objective(trial):
    # Hyperparameters to optimize
    k = trial.suggest_int('k', 20, 100)
    min_k = trial.suggest_int('min_k', 1, 10)
    sim_options = {
        'name': trial.suggest_categorical('sim_options_name', ['msd', 'cosine', 'pearson', 'pearson_baseline']),
        'user_based': trial.suggest_categorical('user_based', [False, True]),
        'min_support': trial.suggest_int('min_support', 1, 5),
    }
    # Additional parameters for pearson_baseline
    if sim_options['name'] == 'pearson_baseline':
        sim_options['shrinkage'] = trial.suggest_int('shrinkage', 0, 200)
    bsl_options = {}
    if sim_options['name'] == 'pearson_baseline':
            # Baseline options are used only with pearson_baseline similarity
        bsl_method = trial.suggest_categorical('bsl_method', ['als', 'sgd'])
        bsl_options['method'] = bsl_method
        bsl_options['n_epochs'] = trial.suggest_int('bsl_n_epochs', 10, 50)  # Default is 10
        if bsl_method == 'als':
            bsl_options['reg_u'] = trial.suggest_float('als_reg_u', 1, 20)  # Default is 15
            bsl_options['reg_i'] = trial.suggest_float('als_reg_i', 1, 20)  # Default is 10
        elif bsl_method == 'sgd':
            bsl_options['reg'] = trial.suggest_float('sgd_reg', 0.01, 0.2)  # Default is 0.02
            bsl_options['learning_rate'] = trial.suggest_float('sgd_learning_rate', 0.001, 0.01)  # Default is 0.005
    # Instantiate the KNNBasic algorithm with the suggested parameters
    algo = KNNBasic(k=k, min_k=min_k, sim_options=sim_options,bsl_options=bsl_options if bsl_options else None)

 
    # Perform cross-validation and return the mean RMSE, MAE, and FCP
    cv_results = cross_validate(algo, data, measures=['RMSE', 'MAE', 'FCP'], cv=5, verbose=False)
    mean_rmse = np.mean(cv_results['test_rmse'])
    mean_mae = np.mean(cv_results['test_mae'])
    mean_fcp = np.mean(cv_results['test_fcp'])

    # Return a dictionary with all the metrics
    return {'rmse': mean_rmse, 'mae': mean_mae, 'fcp': mean_fcp}

study_results = {}
metrics = [ 'rmse','mae','fcp']
directions = [ 'minimize','minimize','maximize']


for metric, direction in zip(metrics, directions):
    study = optuna.create_study(direction=direction, sampler=optuna.samplers.TPESampler(seed=42))
    try:
        if metric == 'rmse':
            study.optimize(lambda trial: knn_basic_objective(trial)['rmse'], n_trials=100, n_jobs=-1)
        elif metric == 'mae':
            study.optimize(lambda trial: knn_basic_objective(trial)['mae'], n_trials=100, n_jobs=-1)
        elif metric == 'fcp':
            study.optimize(lambda trial: knn_basic_objective(trial)['fcp'], n_trials=100, n_jobs=-1)
    except ZeroDivisionError as e:
        print(f"Encountered zero division error during optimization for {metric}: {e}")
    


    # Save the best parameters and best score to a text file
    with open(f'best_params_KNNBasic_{metric}.txt', 'w') as f:
        f.write(f"KNNBasic")
        f.write(f"Best parameters for {metric.upper()}:\n{study.best_params}\n")
        f.write(f"Best {metric.upper()} score: {study.best_value}\n")

[I 2023-11-10 01:40:34,106] A new study created in memory with name: no-name-4aace398-ba70-4192-91f2-23631639ede1


Computing the msd similarity matrix...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Done computing similarity matrix.Computing the cosine similarity matrix...

Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Computing the cosine similarity matrix...
Computing the pearson similarity matrix...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Computing the pearson similarity matrix...
Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Computing the msd similarity matrix...
Done computing

[I 2023-11-10 01:42:18,568] Trial 2 finished with value: 1.7191000697082146 and parameters: {'k': 20, 'min_k': 7, 'sim_options_name': 'cosine', 'user_based': True, 'min_support': 4}. Best is trial 2 with value: 1.7191000697082146.


Computing the pearson_baseline similarity matrix...
Estimating biases using sgd...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...Computing the cosine similarity matrix...

Computing the cosine similarity matrix...
Done computing similarity matrix.


[I 2023-11-10 01:42:29,285] Trial 5 finished with value: 1.7173007948104186 and parameters: {'k': 54, 'min_k': 5, 'sim_options_name': 'pearson_baseline', 'user_based': True, 'min_support': 5, 'shrinkage': 151, 'bsl_method': 'als', 'bsl_n_epochs': 12, 'als_reg_u': 8.581482133303517, 'als_reg_i': 10.944519975103457}. Best is trial 5 with value: 1.7173007948104186.


...

[I 2023-11-10 03:21:44,565] Trial 96 finished with value: 0.17757602284959506 and parameters: {'k': 70, 'min_k': 6, 'sim_options_name': 'cosine', 'user_based': False, 'min_support': 5}. Best is trial 1 with value: 0.5876483169430762.


Computing the cosine similarity matrix...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.


[I 2023-11-10 03:21:54,588] Trial 89 finished with value: 0.16353853936439205 and parameters: {'k': 72, 'min_k': 3, 'sim_options_name': 'pearson_baseline', 'user_based': False, 'min_support': 5, 'shrinkage': 90, 'bsl_method': 'als', 'bsl_n_epochs': 35, 'als_reg_u': 1.9123550689621567, 'als_reg_i': 4.097181117623748}. Best is trial 1 with value: 0.5876483169430762.
[I 2023-11-10 03:21:55,211] Trial 90 finished with value: 0.17580167595424626 and parameters: {'k': 71, 'min_k': 3, 'sim_options_name': 'pearson_baseline', 'user_based': False, 'min_support': 5, 'shrinkage': 96, 'bsl_method': 'als', 'bsl_n_epochs': 35, 'als_reg_u': 18.06428431819621, 'als_reg_i': 2.321356314256086}. Best is trial 1 with value: 0.5876483169430762.


Computing the cosine similarity matrix...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


[I 2023-11-10 03:22:14,038] Trial 98 finished with value: 0.1828829583963028 and parameters: {'k': 69, 'min_k': 7, 'sim_options_name': 'cosine', 'user_based': False, 'min_support': 5}. Best is trial 1 with value: 0.5876483169430762.
[I 2023-11-10 03:22:14,264] Trial 99 finished with value: 0.17857427668279305 and parameters: {'k': 56, 'min_k': 6, 'sim_options_name': 'cosine', 'user_based': False, 'min_support': 5}. Best is trial 1 with value: 0.5876483169430762.


In [4]:
# Define the objective function for hyperparameter optimization for KNNWithMeans
def knn_WithMeans_objective(trial):
    # Hyperparameters to optimize
    k = trial.suggest_int('k', 20, 100)
    min_k = trial.suggest_int('min_k', 1, 10)
    sim_options = {
        'name': trial.suggest_categorical('sim_options_name', ['msd', 'cosine', 'pearson', 'pearson_baseline']),
        'user_based': trial.suggest_categorical('user_based', [False, True]),
        'min_support': trial.suggest_int('min_support', 1, 5),
    }
    # Additional parameters for pearson_baseline
    if sim_options['name'] == 'pearson_baseline':
        sim_options['shrinkage'] = trial.suggest_int('shrinkage', 0, 200)
    bsl_options = {}
    if sim_options['name'] == 'pearson_baseline':
            # Baseline options are used only with pearson_baseline similarity
        bsl_method = trial.suggest_categorical('bsl_method', ['als', 'sgd'])
        bsl_options['method'] = bsl_method
        bsl_options['n_epochs'] = trial.suggest_int('bsl_n_epochs', 10, 50)  # Default is 10
        if bsl_method == 'als':
            bsl_options['reg_u'] = trial.suggest_float('als_reg_u', 1, 20)  # Default is 15
            bsl_options['reg_i'] = trial.suggest_float('als_reg_i', 1, 20)  # Default is 10
        elif bsl_method == 'sgd':
            bsl_options['reg'] = trial.suggest_float('sgd_reg', 0.01, 0.2)  # Default is 0.02
            bsl_options['learning_rate'] = trial.suggest_float('sgd_learning_rate', 0.001, 0.01)  # Default is 0.005
    # Instantiate the KNNWithMeans algorithm with the suggested parameters
    algo = KNNWithMeans(k=k, min_k=min_k, sim_options=sim_options,bsl_options=bsl_options if bsl_options else None)

   
    # Perform cross-validation and return the mean RMSE, MAE, and FCP
    cv_results = cross_validate(algo, data, measures=['RMSE', 'MAE', 'FCP'], cv=5, verbose=False)
    mean_rmse = np.mean(cv_results['test_rmse'])
    mean_mae = np.mean(cv_results['test_mae'])
    mean_fcp = np.mean(cv_results['test_fcp'])

    # Return a dictionary with all the metrics
    return {'rmse': mean_rmse, 'mae': mean_mae, 'fcp': mean_fcp}

study_results = {}
metrics = [ 'rmse','mae','fcp']
directions = [ 'minimize','minimize','maximize']


for metric, direction in zip(metrics, directions):
    study = optuna.create_study(direction=direction, sampler=optuna.samplers.TPESampler(seed=42))
    try:
        if metric == 'rmse':
            study.optimize(lambda trial: knn_WithMeans_objective(trial)['rmse'], n_trials=100, n_jobs=-1)
        elif metric == 'mae':
            study.optimize(lambda trial: knn_WithMeans_objective(trial)['mae'], n_trials=100, n_jobs=-1)
        elif metric == 'fcp':
            study.optimize(lambda trial: knn_WithMeans_objective(trial)['fcp'], n_trials=100, n_jobs=-1)
    except ZeroDivisionError as e:
        print(f"Encountered zero division error during optimization for {metric}: {e}")
    


    # Save the best parameters and best score to a text file
    with open(f'best_params_KNNWithMeans_{metric}.txt', 'w') as f:
        f.write(f"KNNWithMeans")
        f.write(f"Best parameters for {metric.upper()}:\n{study.best_params}\n")
        f.write(f"Best {metric.upper()} score: {study.best_value}\n")

[I 2023-11-10 03:22:14,330] A new study created in memory with name: no-name-0cdde9ac-ac64-4a35-87e9-1e7560bed9a6


Computing the pearson similarity matrix...
Computing the msd similarity matrix...
Estimating biases using sgd...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Computing the cosine similarity matrix...
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Estimating biases using als...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Computing the pearson similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Computing the pears

[I 2023-11-10 03:23:24,684] Trial 1 finished with value: 1.5258263306712994 and parameters: {'k': 88, 'min_k': 8, 'sim_options_name': 'pearson', 'user_based': True, 'min_support': 2}. Best is trial 1 with value: 1.5258263306712994.


Computing the cosine similarity matrix...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Computing the msd similarity matrix...
Estimating biases using als...
Done computing similarity matrix.
Computing the msd similarity matrix...


[I 2023-11-10 03:23:28,041] Trial 2 finished with value: 1.5405886230445285 and parameters: {'k': 75, 'min_k': 2, 'sim_options_name': 'msd', 'user_based': True, 'min_support': 4}. Best is trial 1 with value: 1.5258263306712994.


...

[I 2023-11-10 04:27:26,724] Trial 91 finished with value: 0.5536827353049377 and parameters: {'k': 34, 'min_k': 1, 'sim_options_name': 'cosine', 'user_based': False, 'min_support': 1}. Best is trial 56 with value: 0.5603741905580149.
[I 2023-11-10 04:27:29,373] Trial 94 finished with value: 0.5536823133336505 and parameters: {'k': 31, 'min_k': 5, 'sim_options_name': 'cosine', 'user_based': False, 'min_support': 1}. Best is trial 56 with value: 0.5603741905580149.
[I 2023-11-10 04:27:30,125] Trial 97 finished with value: 0.5504583319838611 and parameters: {'k': 78, 'min_k': 5, 'sim_options_name': 'cosine', 'user_based': False, 'min_support': 1}. Best is trial 56 with value: 0.5603741905580149.
[I 2023-11-10 04:27:30,555] Trial 96 finished with value: 0.5554351979184722 and parameters: {'k': 31, 'min_k': 5, 'sim_options_name': 'cosine', 'user_based': False, 'min_support': 1}. Best is trial 56 with value: 0.5603741905580149.


Computing the cosine similarity matrix...


[I 2023-11-10 04:27:34,770] Trial 95 finished with value: 0.5539571029541994 and parameters: {'k': 33, 'min_k': 5, 'sim_options_name': 'cosine', 'user_based': False, 'min_support': 1}. Best is trial 56 with value: 0.5603741905580149.


Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


[I 2023-11-10 04:27:49,867] Trial 99 finished with value: 0.5565361516113857 and parameters: {'k': 77, 'min_k': 5, 'sim_options_name': 'cosine', 'user_based': False, 'min_support': 1}. Best is trial 56 with value: 0.5603741905580149.
[I 2023-11-10 04:27:50,484] Trial 98 finished with value: 0.5555243459068189 and parameters: {'k': 78, 'min_k': 5, 'sim_options_name': 'cosine', 'user_based': False, 'min_support': 1}. Best is trial 56 with value: 0.5603741905580149.


In [5]:
# Define the objective function for hyperparameter optimization for KNNWithZScore
def knn_WithZScore_objective(trial):
    # Hyperparameters to optimize
    k = trial.suggest_int('k', 20, 100)
    min_k = trial.suggest_int('min_k', 1, 10)
    sim_options = {
        'name': trial.suggest_categorical('sim_options_name', ['msd', 'cosine', 'pearson', 'pearson_baseline']),
        'user_based': trial.suggest_categorical('user_based', [False, True]),
        'min_support': trial.suggest_int('min_support', 1, 5),
    }
    # Additional parameters for pearson_baseline
    if sim_options['name'] == 'pearson_baseline':
        sim_options['shrinkage'] = trial.suggest_int('shrinkage', 0, 200)
    bsl_options = {}
    if sim_options['name'] == 'pearson_baseline':
            # Baseline options are used only with pearson_baseline similarity
        bsl_method = trial.suggest_categorical('bsl_method', ['als', 'sgd'])
        bsl_options['method'] = bsl_method
        bsl_options['n_epochs'] = trial.suggest_int('bsl_n_epochs', 10, 50)  # Default is 10
        if bsl_method == 'als':
            bsl_options['reg_u'] = trial.suggest_float('als_reg_u', 1, 20)  # Default is 15
            bsl_options['reg_i'] = trial.suggest_float('als_reg_i', 1, 20)  # Default is 10
        elif bsl_method == 'sgd':
            bsl_options['reg'] = trial.suggest_float('sgd_reg', 0.01, 0.2)  # Default is 0.02
            bsl_options['learning_rate'] = trial.suggest_float('sgd_learning_rate', 0.001, 0.01)  # Default is 0.005
    # Instantiate the KNNWithZScore algorithm with the suggested parameters
    algo = KNNWithZScore(k=k, min_k=min_k, sim_options=sim_options,bsl_options=bsl_options if bsl_options else None)

    
    # Perform cross-validation and return the mean RMSE, MAE, and FCP
    cv_results = cross_validate(algo, data, measures=['RMSE', 'MAE', 'FCP'], cv=5, verbose=False)
    mean_rmse = np.mean(cv_results['test_rmse'])
    mean_mae = np.mean(cv_results['test_mae'])
    mean_fcp = np.mean(cv_results['test_fcp'])

    # Return a dictionary with all the metrics
    return {'rmse': mean_rmse, 'mae': mean_mae, 'fcp': mean_fcp}

study_results = {}
metrics = [ 'rmse','mae','fcp']
directions = [ 'minimize','minimize','maximize']


for metric, direction in zip(metrics, directions):
    study = optuna.create_study(direction=direction, sampler=optuna.samplers.TPESampler(seed=42))
    try:
        if metric == 'rmse':
            study.optimize(lambda trial: knn_WithZScore_objective(trial)['rmse'], n_trials=100, n_jobs=-1)
        elif metric == 'mae':
            study.optimize(lambda trial: knn_WithZScore_objective(trial)['mae'], n_trials=100, n_jobs=-1)
        elif metric == 'fcp':
            study.optimize(lambda trial: knn_WithZScore_objective(trial)['fcp'], n_trials=100, n_jobs=-1)
    except ZeroDivisionError as e:
        print(f"Encountered zero division error during optimization for {metric}: {e}")
    


    # Save the best parameters and best score to a text file
    with open(f'best_params_KNNWithZScore_{metric}.txt', 'w') as f:
        f.write(f"KNNWithZScore")
        f.write(f"Best parameters for {metric.upper()}:\n{study.best_params}\n")
        f.write(f"Best {metric.upper()} score: {study.best_value}\n")

[I 2023-11-10 04:27:50,549] A new study created in memory with name: no-name-634218e7-09cb-4f32-864b-931ed58bc777


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...Computing the cosine similarity matrix...

Done computing similarity matrix.
Done computing similarity matrix.
Estimating biases using sgd...
Computing the msd similarity matrix...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...

Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing si

[I 2023-11-10 04:28:52,063] Trial 2 finished with value: 1.525354325327318 and parameters: {'k': 81, 'min_k': 5, 'sim_options_name': 'msd', 'user_based': True, 'min_support': 2}. Best is trial 2 with value: 1.525354325327318.


Done computing similarity matrix.
Computing the cosine similarity matrix...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Computing the pearson similarity matrix...
Computing the cosine similarity matrix...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...

Done computing similarity matrix.
Done computing similarity matrix.
Com

[I 2023-11-10 04:29:27,436] Trial 3 finished with value: 1.5261941929318874 and parameters: {'k': 43, 'min_k': 9, 'sim_options_name': 'pearson', 'user_based': True, 'min_support': 3}. Best is trial 2 with value: 1.525354325327318.


...

[I 2023-11-10 05:38:14,245] Trial 97 finished with value: 0.5462991649818498 and parameters: {'k': 92, 'min_k': 3, 'sim_options_name': 'pearson_baseline', 'user_based': False, 'min_support': 1, 'shrinkage': 7, 'bsl_method': 'sgd', 'bsl_n_epochs': 30, 'sgd_reg': 0.07677141102912784, 'sgd_learning_rate': 0.0010586230955351586}. Best is trial 60 with value: 0.5641214850739468.
[I 2023-11-10 05:38:14,309] Trial 96 finished with value: 0.5477082108406378 and parameters: {'k': 90, 'min_k': 3, 'sim_options_name': 'pearson_baseline', 'user_based': False, 'min_support': 1, 'shrinkage': 30, 'bsl_method': 'sgd', 'bsl_n_epochs': 49, 'sgd_reg': 0.0771831764286777, 'sgd_learning_rate': 0.0011456913190652064}. Best is trial 60 with value: 0.5641214850739468.


Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...


[I 2023-11-10 05:38:24,740] Trial 99 finished with value: 0.5464289161655523 and parameters: {'k': 92, 'min_k': 5, 'sim_options_name': 'pearson_baseline', 'user_based': False, 'min_support': 1, 'shrinkage': 11, 'bsl_method': 'sgd', 'bsl_n_epochs': 29, 'sgd_reg': 0.0681343103118527, 'sgd_learning_rate': 0.0011365321710899165}. Best is trial 60 with value: 0.5641214850739468.


Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


[I 2023-11-10 05:38:29,545] Trial 98 finished with value: 0.5428277048831754 and parameters: {'k': 91, 'min_k': 5, 'sim_options_name': 'pearson_baseline', 'user_based': False, 'min_support': 1, 'shrinkage': 9, 'bsl_method': 'sgd', 'bsl_n_epochs': 49, 'sgd_reg': 0.07886509722169055, 'sgd_learning_rate': 0.0016629272791848877}. Best is trial 60 with value: 0.5641214850739468.


In [6]:
# Define the objective function for hyperparameter optimization for KNNBaseline
def knn_Baseline_objective(trial):
    # Hyperparameters to optimize
    k = trial.suggest_int('k', 20, 100)
    min_k = trial.suggest_int('min_k', 1, 10)
    sim_options = {
        'name': trial.suggest_categorical('sim_options_name', ['msd', 'cosine', 'pearson', 'pearson_baseline']),
        'user_based': trial.suggest_categorical('user_based', [False, True]),
        'min_support': trial.suggest_int('min_support', 1, 5),
    }
    # Additional parameters for pearson_baseline
    if sim_options['name'] == 'pearson_baseline':
        sim_options['shrinkage'] = trial.suggest_int('shrinkage', 0, 200)
    bsl_options = {}
    
            # Baseline options are used only with pearson_baseline similarity
    bsl_method = trial.suggest_categorical('bsl_method', ['als', 'sgd'])
    bsl_options['method'] = bsl_method
    bsl_options['n_epochs'] = trial.suggest_int('bsl_n_epochs', 10, 50)  # Default is 10
    if bsl_method == 'als':
        bsl_options['reg_u'] = trial.suggest_float('als_reg_u', 1, 20)  # Default is 15
        bsl_options['reg_i'] = trial.suggest_float('als_reg_i', 1, 20)  # Default is 10
    elif bsl_method == 'sgd':
        bsl_options['reg'] = trial.suggest_float('sgd_reg', 0.01, 0.2)  # Default is 0.02
        bsl_options['learning_rate'] = trial.suggest_float('sgd_learning_rate', 0.001, 0.01)  # Default is 0.005
    # Instantiate the KNNBaseline algorithm with the suggested parameters
    algo = KNNBaseline(k=k, min_k=min_k, sim_options=sim_options,bsl_options=bsl_options if bsl_options else None)


    # Perform cross-validation and return the mean RMSE, MAE, and FCP
    cv_results = cross_validate(algo, data, measures=['RMSE', 'MAE', 'FCP'], cv=5, verbose=False)
    mean_rmse = np.mean(cv_results['test_rmse'])
    mean_mae = np.mean(cv_results['test_mae'])
    mean_fcp = np.mean(cv_results['test_fcp'])

    # Return a dictionary with all the metrics
    return {'rmse': mean_rmse, 'mae': mean_mae, 'fcp': mean_fcp}

study_results = {}
metrics = [ 'rmse','mae','fcp']
directions = [ 'minimize','minimize','maximize']


for metric, direction in zip(metrics, directions):
    study = optuna.create_study(direction=direction, sampler=optuna.samplers.TPESampler(seed=42))
    try:
        if metric == 'rmse':
            study.optimize(lambda trial: knn_Baseline_objective(trial)['rmse'], n_trials=100, n_jobs=-1)
        elif metric == 'mae':
            study.optimize(lambda trial: knn_Baseline_objective(trial)['mae'], n_trials=100, n_jobs=-1)
        elif metric == 'fcp':
            study.optimize(lambda trial: knn_Baseline_objective(trial)['fcp'], n_trials=100, n_jobs=-1)
    except ZeroDivisionError as e:
        print(f"Encountered zero division error during optimization for {metric}: {e}")
    


    # Save the best parameters and best score to a text file
    with open(f'best_params_KNNBaseline_{metric}.txt', 'w') as f:
        f.write(f"KNNBaseline")
        f.write(f"Best parameters for {metric.upper()}:\n{study.best_params}\n")
        f.write(f"Best {metric.upper()} score: {study.best_value}\n")

[I 2023-11-10 05:38:29,616] A new study created in memory with name: no-name-8359861f-f29b-4682-9539-ae31a9a8e120


Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using sgd...
Estimating biases using als...
Computing the msd similarity matrix...
Computing the cosine similarity matrix...
Computing the cosine similarity matrix...
Computing the msd similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Computing the msd similarity matrix...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases usingEstimating biases using sgd...
 als...
Estimating biases using sgd...
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.Computing the cosine similarity matrix...

Estimating biases using 

[I 2023-11-10 05:39:05,197] Trial 1 finished with value: 1.4991962355790638 and parameters: {'k': 55, 'min_k': 5, 'sim_options_name': 'cosine', 'user_based': True, 'min_support': 2, 'bsl_method': 'sgd', 'bsl_n_epochs': 11, 'sgd_reg': 0.13441980564884048, 'sgd_learning_rate': 0.0033558089965056094}. Best is trial 1 with value: 1.4991962355790638.


Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Estimating biases using als...
Estimating biases using sgd...
Computing the msd similarity matrix...
Estimating biases using als...
Estimating biases using als...
Estimating biases using sgd...
Computing the msd similarity matrix...
Estimating biases using als...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Estimating biases using als...
Computing the msd similarity matrix...
Estimating biases using als...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done co

[I 2023-11-10 05:39:20,487] Trial 4 finished with value: 1.6441705951998018 and parameters: {'k': 76, 'min_k': 1, 'sim_options_name': 'msd', 'user_based': True, 'min_support': 1, 'bsl_method': 'als', 'bsl_n_epochs': 19, 'als_reg_u': 9.092083007997052, 'als_reg_i': 3.148507590487666}. Best is trial 1 with value: 1.4991962355790638.


...

[I 2023-11-10 07:34:50,896] Trial 95 finished with value: 0.5762730761254284 and parameters: {'k': 38, 'min_k': 4, 'sim_options_name': 'pearson_baseline', 'user_based': False, 'min_support': 1, 'shrinkage': 69, 'bsl_method': 'als', 'bsl_n_epochs': 24, 'als_reg_u': 11.512829260466246, 'als_reg_i': 4.23890300901593}. Best is trial 87 with value: 0.577698054539162.


Computing the pearson_baseline similarity matrix...


[I 2023-11-10 07:34:55,217] Trial 96 finished with value: 0.5781573138466817 and parameters: {'k': 85, 'min_k': 4, 'sim_options_name': 'pearson_baseline', 'user_based': False, 'min_support': 1, 'shrinkage': 62, 'bsl_method': 'als', 'bsl_n_epochs': 24, 'als_reg_u': 7.106894006905868, 'als_reg_i': 2.293850363034217}. Best is trial 96 with value: 0.5781573138466817.


Done computing similarity matrix.
Estimating biases using als...
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


[I 2023-11-10 07:35:04,869] Trial 98 finished with value: 0.572811303630336 and parameters: {'k': 83, 'min_k': 4, 'sim_options_name': 'pearson_baseline', 'user_based': False, 'min_support': 1, 'shrinkage': 71, 'bsl_method': 'als', 'bsl_n_epochs': 24, 'als_reg_u': 7.248000468009109, 'als_reg_i': 2.031933442788861}. Best is trial 96 with value: 0.5781573138466817.
[I 2023-11-10 07:35:05,645] Trial 99 finished with value: 0.5741224310967661 and parameters: {'k': 39, 'min_k': 4, 'sim_options_name': 'pearson_baseline', 'user_based': False, 'min_support': 1, 'shrinkage': 63, 'bsl_method': 'als', 'bsl_n_epochs': 24, 'als_reg_u': 6.6998720897687765, 'als_reg_i': 2.3279181468278387}. Best is trial 96 with value: 0.5781573138466817.
[I 2023-11-10 07:35:06,149] Trial 97 finished with value: 0.5754571280557362 and parameters: {'k': 83, 'min_k': 4, 'sim_options_name': 'pearson_baseline', 'user_based': False, 'min_support': 1, 'shrinkage': 66, 'bsl_method': 'als', 'bsl_n_epochs': 24, 'als_reg_u': 7.